In [ ]:
import os
os.environ['LECTURE_NUMBER'] = '28'

# Linear Regression

In [ ]:
from datascience import * 
from cs104 import *
import numpy as np
%matplotlib inline

In [ ]:
# HIDE_INPUT

def plot_line(a, b, x): 
    """
    Plots a line using the give slope and intercept over the
    range of x-values in the x array.
    """
    xlims = make_array(min(x), max(x))
    plot = Plot()
    plot.line(xlims, a * xlims + b, lw=4, clip_on=False)

def line_predictions(a, b, x): 
    """
    Computes the prediction  y_hat = a * x + b
    where a and b are the slope and intercept and x is the set of x-values.
    """
    return a * x + b

def visualize_scatter_with_line_and_residuals(table, x_label, y_label, a, b):
    """
    Draw a scatter plot, a line, and the line segments capturing the residuals 
    for some of the points.
    """
    y_hat = line_predictions(a, b, table.column(x_label))
    prediction_table = table.with_columns(y_label + '_hat', y_hat)
    plot = table.scatter(x_label, y_label, title='a = ' + str(round(a,3)) + '; b = ' + str(round(b,3)))
    xlims = make_array(min(table.column(x_label)), max(table.column(x_label)))
    plot.line(xlims, a * xlims + b, lw=4, color="C0")
    
    every10th = prediction_table.sort(x_label).take(np.arange(0, 
                                                           prediction_table.num_rows, 
                                                           prediction_table.num_rows // 10))
    for row in every10th.rows:
        x = row.item(x_label)
        y = row.item(y_label)
        plot.line([x, x], [y, a * x + b], color='r', lw=2)    
        plot.dot(x,y,color='C0',s=70)

    return plot

def mean_squared_error(table, x_label, y_label, a, b): 
    y_hat = line_predictions(a, b, table.column(x_label))
    residual = table.column(y_label) - y_hat
    return np.mean(residual**2)

def visualize_scatter_with_line_and_mse(table, x_label, y_label, a, b):
    plot = visualize_scatter_with_line_and_residuals(table, x_label, y_label, a, b)
    mse = mean_squared_error(x_y_table, x_label, y_label, a, b)
    plot.set_title('a = ' + str(round(a,3)) + 
                '; b = ' + str(round(b,3)) + 
                '\n mse = ' + str(round(mse,3)))

    
## MSE HEAT

def plot_regression_line_and_mse_heat(table, x_label, y_label, a, b, show_mse='2d', fig=None):
        """
        Left plot: the scatter plot with line y=ax+b
        Right plot: None, 2D heat map of MSE, or 3D surface plot of MSE 
        """

        a_space = np.linspace(-0.5, 0.5, 200)
        b_space = np.linspace(-35, -8, 200)
        a_space, b_space = np.meshgrid(a_space, b_space)
        broadcasted = np.broadcast(a_space, b_space)
        mses = np.empty(broadcasted.shape)
        mses.flat = [mean_squared_error(table, 'x', 'y', a, b) for (a,b) in broadcasted]
        
        if fig is None:
            fig = Figure(1,2)

        ax = fig.axes()

        #Plot the scatter plot and best fit line on the left
        with Plot(ax[0]):
            plot_scatter_with_line(table, x_label, y_label, a, b)

        if show_mse == '2d':
            with Plot(ax[1]):
                mse = mean_squared_error(table, x_label, y_label, a, b)
                ax[1].pcolormesh(a_space, b_space, mses, cmap='viridis', 
                                 norm=colors.PowerNorm(gamma=0.25,
                                                       vmin=mses.min(), 
                                                       vmax=mses.max()));
                ax[1].scatter(a,b,s=100,color='red');
                ax[1].set_xlabel('a')
                ax[1].set_ylabel('b')
                ax[1].set_title('Mean Squared Error: ' + str(np.round(mse, 3)))

        elif show_mse == "3d": 
            ax[1] = plots.subplot(1, 2, 2, projection='3d')
            with Plot(ax[1]):
                ax[1].plot_surface(a_space, b_space, mses,
                                cmap='viridis', 
                                antialiased=False, 
                                linewidth=0,
                                norm = colors.PowerNorm(gamma=0.25,vmin=mses.min(), vmax=mses.max()))

                ax[1].plot([a],[b],[mean_squared_error(table, x_label, y_label, a, b)], 'ro',zorder=3);
                ax[1].set_xlabel('a')
                ax[1].set_ylabel('b')
                ax[1].set_title('Mean Squared Error')      

def visualize_regression_line_and_mse_heat(table, x_label, y_label, show_mse=None):
    
    a_space = np.linspace(-0.5, 0.5, 200)
    b_space = np.linspace(-35, -8, 200)
    a_space, b_space = np.meshgrid(a_space, b_space)
    broadcasted = np.broadcast(a_space, b_space)
    mses = np.empty(broadcasted.shape)
    mses.flat = [mean_squared_error(table, 'x', 'y', a, b) for (a,b) in broadcasted]
    
    def visualize_regression_line_and_mse_heat_helper(a, b, show_mse=show_mse, fig=None):
        """
        Left plot: the scatter plot with line y=ax+b
        Right plot: None, 2D heat map of MSE, or 3D surface plot of MSE 
        """

        if fig is None:
            fig = Figure(1,2)

        ax = fig.axes()

        #Plot the scatter plot and best fit line on the left
        with Plot(ax[0]):
            plot_scatter_with_line(table, x_label, y_label, a, b)

        if show_mse == '2d':
            with Plot(ax[1]):
                mse = mean_squared_error(table, x_label, y_label, a, b)
                ax[1].pcolormesh(a_space, b_space, mses, cmap='viridis', 
                                 norm=colors.PowerNorm(gamma=0.25,
                                                       vmin=mses.min(), 
                                                       vmax=mses.max()));
                ax[1].scatter(a,b,s=100,color='red');
                ax[1].set_xlabel('a')
                ax[1].set_ylabel('b')
                ax[1].set_title('Mean Squared Error: ' + str(np.round(mse, 3)))

        elif show_mse == "3d": 
            ax[1] = plots.subplot(1, 2, 2, projection='3d')
            with Plot(ax[1]):
                ax[1].plot_surface(a_space, b_space, mses,
                                cmap='viridis', 
                                antialiased=False, 
                                linewidth=0,
                                norm = colors.PowerNorm(gamma=0.25,vmin=mses.min(), vmax=mses.max()))

                ax[1].plot([a],[b],[mean_squared_error(table, x_label, y_label, a, b)], 'ro',zorder=3);
                ax[1].set_xlabel('a')
                ax[1].set_ylabel('b')
                ax[1].set_title('Mean Squared Error')
            
    interact(visualize_regression_line_and_mse_heat_helper, 
                a = Slider(-0.5,0.5,0.01),
                b = Slider(-35,-8,0.1),
                fig = Fixed(None),
                show_mse=Choice([None, "2d", "3d"]))            

## 1. Lines on scatter plots

In [ ]:
#load data from the lecture 7 and clean it up 
greenland_climate = Table().read_table('../../../lectures/data/climate_upernavik.csv')
greenland_climate = greenland_climate.relabeled('Precipitation (millimeters)', 
                                                "Precip (mm)")
tidy_greenland = greenland_climate.where('Air temperature (C)', 
                                         are.not_equal_to(999.99))
tidy_greenland = tidy_greenland.where('Sea level pressure (mbar)', 
                                      are.not_equal_to(9999.9))
feb = tidy_greenland.where('Month', are.equal_to(2))


In [ ]:
x = feb.column('Year') - 1880 
y = feb.column('Air temperature (C)')
x_y_table = Table().with_columns("x", x, "y", y)

In [ ]:
html_interact(plot_scatter_with_line, 
         table = Fixed(x_y_table),
         x_label = Fixed('x'),
         y_label = Fixed('y'),
        a = Slider(-0.5,0.5,0.125),
         b = Slider(-35,-8,0.02))

## 2. Predict and evaluate

In [ ]:
html_interact(visualize_scatter_with_line_and_residuals, 
        table = Fixed(x_y_table),
        x_label = Fixed('x'),
        y_label = Fixed('y'),                     
        a = Slider(-0.5,0.5,0.125),
        b = Slider(-35,-8,2))

## 3. Fit the Best Line Manually

In [ ]:
# REMOVE_INPUT
def visualize_regression_line_and_mse_heat(table, x_label, y_label, show_mse=None):
    
    a_space = np.linspace(-0.5, 0.5, 200)
    b_space = np.linspace(-35, -8, 200)
    a_space, b_space = np.meshgrid(a_space, b_space)
    broadcasted = np.broadcast(a_space, b_space)
    mses = np.empty(broadcasted.shape)
    mses.flat = [mean_squared_error(table, 'x', 'y', a, b) for (a,b) in broadcasted]
    
    def visualize_regression_line_and_mse_heat_helper(a, b, show_mse=show_mse):
        """
        Left plot: the scatter plot with line y=ax+b
        Right plot: None, 2D heat map of MSE, or 3D surface plot of MSE 
        """

        fig = Figure(1,2)

        ax = fig.axes()

        #Plot the scatter plot and best fit line on the left
        with Plot(ax[0]):
            plot_scatter_with_line(table, x_label, y_label, a, b)

        if show_mse == '2d':
            with Plot(ax[1]):
                mse = mean_squared_error(table, x_label, y_label, a, b)
                ax[1].pcolormesh(a_space, b_space, mses, cmap='viridis', 
                                 norm=colors.PowerNorm(gamma=0.25,
                                                       vmin=mses.min(), 
                                                       vmax=mses.max()));
                ax[1].scatter(a,b,s=100,color='red');
                ax[1].set_xlabel('a')
                ax[1].set_ylabel('b')
                ax[1].set_title('Mean Squared Error: ' + str(np.round(mse, 3)))

        elif show_mse == "3d": 
            ax[1] = plots.subplot(1, 2, 2, projection='3d')
            with Plot(ax[1]):
                ax[1].plot_surface(a_space, b_space, mses,
                                cmap='viridis', 
                                antialiased=False, 
                                linewidth=0,
                                norm = colors.PowerNorm(gamma=0.25,vmin=mses.min(), vmax=mses.max()))

                ax[1].plot([a],[b],[mean_squared_error(table, x_label, y_label, a, b)], 'ro',zorder=3);
                ax[1].set_xlabel('a')
                ax[1].set_ylabel('b')
                ax[1].set_title('Mean Squared Error')
                
        return fig
            
    return html_interact(visualize_regression_line_and_mse_heat_helper, 
                a = Slider(-0.25,0.25,0.125),
                b = Slider(-30,-15,2),
                show_mse=Choice(["2d", "3d"]))            

visualize_regression_line_and_mse_heat(x_y_table, 'x', 'y')